# Model 3 Random Forests

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

## Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select your features (columns)

In [5]:
# Get rid of unnecessary columns (columns that are not independently measured)
# Set features. This will also be used as your x values.
# Assign X (data) and y (target)

data = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 
         'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 
         'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 
         'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad', 
         'ra', 'dec', 'koi_kepmag']]
feature_names = data.columns

target = df["koi_disposition"]
target_names = ["confirmed", "false positive", "candidate"]

data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,6046,4.486,0.972,296.28613,48.224670,15.714


## Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,292.16705,48.727589,15.263


## Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

## Create and Fit the Model

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, encoded_y_train)

In [10]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
# Sort the features by their importance
sorted(zip(rf.feature_importances_, data.columns), reverse=True)

[(0.1330412469704479, 'koi_model_snr'),
 (0.12912775516230365, 'koi_fpflag_nt'),
 (0.12445768086268622, 'koi_fpflag_co'),
 (0.09534851165588523, 'koi_fpflag_ss'),
 (0.0726971345950565, 'koi_prad'),
 (0.054815723578201365, 'koi_depth'),
 (0.050489400935595306, 'koi_period'),
 (0.04675326565707169, 'koi_fpflag_ec'),
 (0.04203871880535739, 'koi_impact'),
 (0.031868622692840215, 'koi_teq'),
 (0.03134469765691763, 'koi_duration'),
 (0.029977367693873873, 'koi_insol'),
 (0.02851714012813965, 'koi_time0bk'),
 (0.024532001456906503, 'koi_steff'),
 (0.022402826542728127, 'ra'),
 (0.021171781630664844, 'dec'),
 (0.02097926335932868, 'koi_srad'),
 (0.020922480900138263, 'koi_kepmag'),
 (0.01951437971585706, 'koi_slogg')]

In [11]:
# Set significant features
data = df[['koi_fpflag_nt', 'koi_model_snr', 'koi_fpflag_co', 'koi_fpflag_ss', 
         'koi_prad', 'koi_depth', 'koi_period', 'koi_fpflag_ec', 
         'koi_impact', 'koi_teq']]
feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_model_snr,koi_fpflag_co,koi_fpflag_ss,koi_prad,koi_depth,koi_period,koi_fpflag_ec,koi_impact,koi_teq
0,0,25.8,0,0,2.83,874.8,54.418383,0,0.586,443
1,0,76.3,0,1,14.60,10829.0,19.899140,0,0.969,638
2,0,505.6,0,1,33.46,8079.2,1.736952,0,1.276,1395
3,0,40.9,0,0,2.75,603.3,2.525592,0,0.701,1406
4,0,40.2,0,0,2.77,686.0,4.134435,0,0.762,1160


## Train the Model



In [12]:
# Recreate train test split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

# Rescale data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Retrain model
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, encoded_y_train)

In [13]:
print(f"Training Data Score: {rf.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, encoded_y_test)}")

from sklearn.metrics import classification_report

predictions = rf.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

Training Data Score: 1.0
Testing Data Score: 0.8924485125858124
                precision    recall  f1-score   support

     confirmed       0.81      0.74      0.77       411
false positive       0.80      0.83      0.82       484
     candidate       0.98      1.00      0.99       853

      accuracy                           0.89      1748
     macro avg       0.86      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [400], 
              'max_features' : ['auto', 'sqrt'],
              'min_samples_split': [2, 4]}

grid = GridSearchCV(rf, param_grid, verbose=3)

In [15]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.908 total time=   3.8s
[CV 2/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.897 total time=   4.0s
[CV 3/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.880 total time=   4.5s
[CV 4/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.886 total time=   4.2s
[CV 5/5] END max_features=auto, min_samples_split=2, n_estimators=400;, score=0.882 total time=   4.3s
[CV 1/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.907 total time=   4.2s
[CV 2/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.896 total time=   4.2s
[CV 3/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.882 total time=   4.2s
[CV 4/5] END max_features=auto, min_samples_split=4, n_estimators=400;, score=0.881 total time=   4.1s
[CV 5/5] END 

GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [2, 4], 'n_estimators': [400]},
             verbose=3)

In [16]:
print(grid.best_params_)
print(grid.best_score_)

{'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 400}
0.8909000938734819


In [17]:
# Make predictions with hypertuned model
predictions = grid.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

                precision    recall  f1-score   support

     confirmed       0.81      0.75      0.78       411
false positive       0.81      0.84      0.82       484
     candidate       0.98      1.00      0.99       853

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'RandomForest.sav'
joblib.dump(rf, filename)

['RandomForest.sav']